In [30]:
import os

def maybe_mkdir_p(directory: str) -> None:
    os.makedirs(directory, exist_ok=True)

maic_dir = '/mnt/backup/'
base_dir = os.path.join(maic_dir, 'working')
input_dir = '/mnt/dataset/'
temp_dir = '/mnt/temp/'

maybe_mkdir_p(base_dir)
maybe_mkdir_p(temp_dir)

# ! git clone https://github.com/keemsir/nnUNet.git

respository_dir = os.path.join(maic_dir, 'nnUNet')
os.chdir(respository_dir)

! pip install -e .

os.chdir(base_dir)

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
Obtaining file:///mnt/backup/nnUNet
  Attempting uninstall: nnunet
    Found existing installation: nnunet 1.7.0
    Can't uninstall 'nnunet'. No files were found to uninstall.
  Running setup.py develop for nnunet
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [31]:
# installed
import shutil
import json
import numpy as np

from collections import OrderedDict
from scipy import special
import copy
import cv2
from tqdm import tqdm
import h5py

# must install
import pydicom
import nibabel as nib
import SimpleITK as sitk

In [3]:
'''
! mkdir -p /root/.pip


%%writefile /root/.pip/pip.conf
[global]
index-url=http://ftp.daumkakao.com/pypi/simple
trusted-host=ftp.daumkakao.com
'''

'\n! mkdir -p /root/.pip\n\n\n%%writefile /root/.pip/pip.conf\n[global]\nindex-url=http://ftp.daumkakao.com/pypi/simple\ntrusted-host=ftp.daumkakao.com\n'

In [27]:
'''
# zipfile open
import zipfile

with zipfile.ZipFile('/tf/backup/nnUNet.zip', 'r') as existing_zip:
#     existing_zip.extractall('/tf/temp/')
    existing_zip.extractall('/tf/submission/submit_files/')
'''

In [32]:
import os

def maybe_mkdir_p(directory: str) -> None:
    os.makedirs(directory, exist_ok=True)

maic_dir = '/mnt/backup/'
base_dir = os.path.join(maic_dir, 'working')
input_dir = '/mnt/dataset/'
temp_dir = '/mnt/temp/'

maybe_mkdir_p(base_dir)
maybe_mkdir_p(temp_dir)

# ! git clone https://github.com/keemsir/nnUNet.git

respository_dir = os.path.join(maic_dir, 'nnUNet')
os.chdir(respository_dir)

! pip install -e .

os.chdir(base_dir)

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
Obtaining file:///mnt/backup/nnUNet
  Attempting uninstall: nnunet
    Found existing installation: nnunet 1.7.0
    Can't uninstall 'nnunet'. No files were found to uninstall.
  Running setup.py develop for nnunet
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [33]:
task_name = 'Task55_PETCT'
convert_name = 'Task555_PETCT'

main_dir = os.path.join(base_dir, 'nnUNet/nnunet')
mainT_dir = os.path.join(temp_dir, 'nnUNet/nnunet')

rawbase_dir = os.path.join(mainT_dir, 'nnUNet_raw_data_base/')

pp_dir = os.path.join(mainT_dir, 'preprocessed')
tasks_dir = os.path.join(mainT_dir, 'Tasks')
task_dir = os.path.join(tasks_dir, task_name)

model_dir = os.path.join(main_dir, 'nnUNet_trained_models')
Prediction_dir = os.path.join(main_dir, 'nnUNet_Prediction_Results')
result_dir = os.path.join(Prediction_dir, convert_name)

staple_dir = os.path.join(Prediction_dir, 'staple')

# 1. Data preprocessing
maybe_mkdir_p(tasks_dir)
maybe_mkdir_p(temp_dir)

# 2. Directory
maybe_mkdir_p(main_dir)
maybe_mkdir_p(model_dir)
maybe_mkdir_p(pp_dir)

# 3. Directory
maybe_mkdir_p(result_dir)
maybe_mkdir_p(staple_dir)


#Environment Setting
os.environ['nnUNet_raw_data_base'] = rawbase_dir #os.path.join(mainT_dir, 'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = pp_dir #os.path.join(mainT_dir, 'preprocessed')
os.environ['RESULTS_FOLDER'] = model_dir #os.path.join(main_dir, 'nnUNet_trained_models')


print('Setting Completed!')

Setting Completed!


# 1-1. Data preprocessing

In [14]:
def hdf2nifti(hdf_folder: str, save_folder: str):
    # hdf_folder : [train_dir, test_dir] hdf5 file path
    # save_folder : [imagesTr, imagesTs] Save Folder path
    maybe_mkdir_p(os.path.join(save_folder, 'imagesTr'))
    maybe_mkdir_p(os.path.join(save_folder, 'labelsTr'))
    print('Creating "{}" Image & Label ..'.format(os.path.basename(os.path.normpath(save_folder))))
    hdf5_files = os.listdir(hdf_folder)


    for hdf5_file in hdf5_files:


        hdf5_path = os.path.join(hdf_folder, hdf5_file)

        # image
        f_i = h5py.File(hdf5_path, 'r')
        ctarr = np.asarray(f_i['CT'])
        petarr = np.asarray(f_i['PET'])
        labels = np.asarray(f_i['Aorta'])
        f_i.close()

        SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT = ctarr.shape
        images = np.empty([SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT, 0], dtype=np.single)

        image_ct = np.expand_dims(ctarr, axis=3)
        images = np.append(images, image_ct, axis=3)
        image_pet = np.expand_dims(petarr, axis=3)
        images = np.append(images, image_pet, axis=3)


        hdf5_file_NAME = hdf5_file

        niim = nib.Nifti1Image(images, affine=np.eye(4))
        nib.save(niim, os.path.join(save_folder, 'imagesTr/{}.nii.gz'.format(hdf5_file[:-8])))

        nila = nib.Nifti1Image(labels, affine=np.eye(4))
        nib.save(nila, os.path.join(save_folder, 'labelsTr/{}.nii.gz'.format(hdf5_file[:-8])))


    print('"{}" Image & Label Completed !!'.format(os.path.basename(os.path.normpath(save_folder))))
    print('Image Patient : {}'.format(len(os.listdir(input_dir))))

hdf2nifti(input_dir, task_dir)

Creating "Task55_PETCT" Image & Label ..
"Task55_PETCT" Image & Label Completed !!
Image Patient : 80


In [15]:
task_dir

'/mnt/temp/nnUNet/nnunet/Tasks/Task55_PETCT'

In [16]:
def json_mk(save_dir: str):
    # Path
    imagesTr = os.path.join(save_dir, 'imagesTr')
    imagesTs = os.path.join(save_dir, 'imagesTs')
    maybe_mkdir_p(imagesTr)
    maybe_mkdir_p(imagesTs)

    overwrite_json_file = True
    json_file_exist = False

    if os.path.exists(os.path.join(save_dir, 'dataset.json')):
        print('dataset.json already exist!')
        json_file_exist = True

    if json_file_exist == False or overwrite_json_file:

        json_dict = OrderedDict()
        json_dict['name'] = "PETCT"
        json_dict['description'] = "Medical Image AI Challenge 2021"
        json_dict['tensorImageSize'] = "4D"
        json_dict['reference'] = "https://maic.or.kr/competitions/"
        json_dict['licence'] = "SNUH"
        json_dict['release'] = "18/10/2021"

        json_dict['modality'] = {
            "0": "CT",
            "1": "PET"
        }
        json_dict['labels'] = {
            "0": "background",
            "1": "Aorta"
        }

        train_ids = sorted(os.listdir(imagesTr))
        test_ids = sorted(os.listdir(imagesTs))
        json_dict['numTraining'] = len(train_ids)
        json_dict['numTest'] = len(test_ids)

        json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

        json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids] #(i[:i.find("_0000")])

        with open(os.path.join(save_dir, "dataset.json"), 'w') as f:
            json.dump(json_dict, f, indent=4, sort_keys=False)

        if os.path.exists(os.path.join(save_dir, 'dataset.json')):
            if json_file_exist == False:
                print('dataset.json created!')
            else:
                print('dataset.json overwritten!')
json_mk(task_dir)

dataset.json created!


# 1-2. Data preprocessing

In [17]:
os.chdir(mainT_dir)
!nnUNet_convert_decathlon_task -i /mnt/temp/nnUNet/nnunet/Tasks/Task55_PETCT -output_task_id 555 # -i : task_dir
!nnUNet_plan_and_preprocess -t 555 # --verify_dataset_integrity
os.chdir(base_dir)



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

23090557_20130
23090560_20160
23090563_20151
23090567_20160
23090571_20120
23090579_20141
23090585_20130
23090582_20150
before crop: (2, 299, 128, 128) after crop: (2, 299, 128, 128) spacing: [1. 1. 1.] 

before crop: (2, 299, 128, 128) after crop: (2, 299, 128, 128) spacing:

# 2. Training

In [50]:
!nnUNet_train 2d nnUNetTrainerV2 555 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'CT', 1: 'PET'}
use_mask_for_norm OrderedDict([(0, False), (1, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'CT'), (1, 'nonCT')])
stages...

stage:  0
{'batch_size': 202, 'num_pool_per_axis': [5, 5], 'patch_size': array([128, 128]), 'median_patient_size_in_voxels': array([299, 12

In [34]:
!nnUNet_train 3d_fullres nnUNetTrainerV2 555 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'CT', 1: 'PET'}
use_mask_for_norm OrderedDict([(0, False), (1, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'CT'), (1, 'nonCT')])
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [5, 4, 4], 'patch_size': array([256,  96,  96]), 'median_patient_size_in_voxels': 

In [59]:
!nnUNet_train 2d nnUNetTrainerV2_Loss_Dice 555 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNet_variants.loss_function.nnUNetTrainerV2_Loss_Dice.nnUNetTrainerV2_Loss_Dice'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'CT', 1: 'PET'}
use_mask_for_norm OrderedDict([(0, False), (1, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'CT'), (1, 'nonCT')])
stages...

stage:  0
{'batch_size': 202, 'num_pool_per_axis': [5, 5], 'patch_size': array([128, 128]

In [9]:
# Model 1
# os.chdir(main_dir)

# !nnUNet_train 2d nnUNetTrainerV2 555 0
!nnUNet_train 2d nnUNetTrainerV2 555 1
!nnUNet_train 2d nnUNetTrainerV2 555 2
!nnUNet_train 2d nnUNetTrainerV2 555 3
!nnUNet_train 2d nnUNetTrainerV2 555 4

!nnUNet_train 3d_fullres nnUNetTrainerV2 555 0
!nnUNet_train 3d_fullres nnUNetTrainerV2 555 1
!nnUNet_train 3d_fullres nnUNetTrainerV2 555 2
!nnUNet_train 3d_fullres nnUNetTrainerV2 555 3
!nnUNet_train 3d_fullres nnUNetTrainerV2 555 4

# os.chdir(base_dir)

# !nnUNet_predict -i /mnt/temp/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs/ -o /mnt/backup/working/nnUNet/nnunet/nnUNet_Prediction_Results/Task555_PETCT/2d_CEGDL/ -t 577 -tr nnUNetTrainerV2_Loss_CEGDL -m 2d --disable_tta



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Traceback (most recent call last):
  File "/usr/local/bin/nnUNet_train", line 33, in <module>
    sys.exit(load_entry_point('nnunet', 'console_scripts', 'nnUNet_train')())
  File "/mnt/backup/nnUNet/nnunet/run/run_training.py", line 137, in main
    trainer_class = get_default_configuration(network, task, network_trainer, plans_identifier)
  File "/mnt/backup/nnUNet/nnunet/run/default_configuration.py", line 47, in get_default_configuration
    plans = load_pickle(plans_file)
  File "/usr/local/lib/python3.8/dist-packages/batchgenerators/utilities/file_and_folder_operations.py", line 57, in load_pickle
    with open(file, mode) as

UnboundLocalError: local variable 'child' referenced before assignment

In [145]:
!nnUNet_predict -i /tf/sample -o /tf/2d_sample/ -t 555 -tr nnUNetTrainerV2 -m 2d --disable_tta
!nnUNet_predict -i /tf/sample -o /tf/3d_sample/ -t 555 -tr nnUNetTrainerV2 -m 3d_fullres --disable_tta



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /mnt/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 2 input modalities for each image
Found 1 unique case ids, here are some examples: ['23010018_20141226']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 1
number of cases that still need to be predicted: 1
emptying cuda cache
loading parameters for folds, None
folds is None so we will automatically look for output folders (not using 'all'!)
found the following folds:  ['/mnt/backup/working/nnUNet/nnunet/nnU

In [71]:
# Model 2
os.chdir(main_dir)

!nnUNet_train 2d nnUNetTrainerV2 555 0
!nnUNet_train 2d nnUNetTrainerV2 555 1
!nnUNet_train 2d nnUNetTrainerV2 555 2
!nnUNet_train 2d nnUNetTrainerV2 555 3
!nnUNet_train 2d nnUNetTrainerV2 555 4

os.chdir(base_dir)



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'CT', 1: 'PET'}
use_mask_for_norm OrderedDict([(0, False), (1, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'CT'), (1, 'nonCT')])
stages...

stage:  0
{'batch_size': 202, 'num_pool_per_axis': [5, 5], 'patch_size': array([128, 128]), 'median_patient_size_in_voxels': array([299, 12

In [88]:
os.listdir('/mnt/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/')
!nnUNet_predict -i /tf/sample_input -o /tf/2d_fold12345/ -t 555 -tr nnUNetTrainerV2 -m 2d --disable_tta



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /mnt/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 2 input modalities for each image
Found 3 unique case ids, here are some examples: ['23010017_20141226' '23010018_20141226' '23010017_20141226']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 3
number of cases that still need to be predicted: 3
emptying cuda cache
loading parameters for folds, None
folds is None so we will automatically look for output folders (not using 'all'!)
found the following folds: 

In [72]:
# Model 3
os.chdir(main_dir)

!nnUNet_train 3d_fullres nnUNetTrainerV2 555 0
!nnUNet_train 3d_fullres nnUNetTrainerV2 555 1
!nnUNet_train 3d_fullres nnUNetTrainerV2 555 2
!nnUNet_train 3d_fullres nnUNetTrainerV2 555 3
!nnUNet_train 3d_fullres nnUNetTrainerV2 555 4

os.chdir(base_dir)



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'CT', 1: 'PET'}
use_mask_for_norm OrderedDict([(0, False), (1, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'CT'), (1, 'nonCT')])
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [5, 4, 4], 'patch_size': array([256,  96,  96]), 'median_patient_size_in_voxels': 

UnboundLocalError: local variable 'child' referenced before assignment

In [52]:
model_1 = os.path.join(model_dir, 'nnUNet/2d/')
# fold_list = ['fold_0', 'fold_1', 'fold_2', 'fold_3', 'fold_4']
# fold_list = [fold_0, fold_1, fold_2, fold_3, fold_4]

fold_all = os.path.join(model_1, 'Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/all')
fold_0 = os.path.join(model_1, 'Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0')
fold_1 = os.path.join(model_1, 'Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/fold_1')
fold_2 = os.path.join(model_1, 'Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/fold_2')
fold_3 = os.path.join(model_1, 'Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/fold_3')
fold_4 = os.path.join(model_1, 'Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/fold_4')

fold_list = [fold_0, fold_1, fold_2, fold_3, fold_4]

In [53]:
for fold in fold_list:
    if os.path.exists(fold):
        shutil.rmtree(fold)
        shutil.copytree(fold_all, fold)
    else:
        shutil.copytree(fold_all, fold)

In [7]:
os.chdir('/tf')

In [1]:
! /mnt/utils/setup.sh

In [9]:
os.getcwd()

'/tf'

In [12]:
os.listdir('/mnt/')

['utils', 'backup', 'dataset', 'submission', 'temp']

In [38]:
os.listdir('/mnt/backup/working/nnUNet/nnunet/nnUNet_Prediction_Results/Task555_PETCT/2d/')

['23010019_20141.nii.gz',
 '23010017_20141.nii.gz',
 '23010018_20141.nii.gz',
 'plans.pkl']

In [21]:
print(os.listdir('/mnt/dataset'))
print(len(os.listdir('/mnt/dataset')))

['23090580_20131226.hdf5', '23090618_20161212.hdf5', '23090628_20150204.hdf5', '23090643_20121227.hdf5', '23090644_20131216.hdf5', '23090636_20121018.hdf5', '23090585_20130213.hdf5', '23090572_20130226.hdf5', '23090603_20141212.hdf5', '23090601_20130225.hdf5', '23090569_20120607.hdf5', '23090599_20140701.hdf5', '23090597_20130227.hdf5', '23090627_20160608.hdf5', '23090559_20150812.hdf5', '23090614_20120402.hdf5', '23090557_20130717.hdf5', '23090634_20150409.hdf5', '23090581_20130626.hdf5', '23090626_20160119.hdf5', '23090568_20121018.hdf5', '23090623_20120406.hdf5', '23090615_20140403.hdf5', '23090640_20140711.hdf5', '23090613_20130208.hdf5', '23090641_20160510.hdf5', '23090609_20120510.hdf5', '23090560_20160114.hdf5', '23090563_20151216.hdf5', '23090596_20150112.hdf5', '23090632_20130807.hdf5', '23090598_20130103.hdf5', '23090584_20120523.hdf5', '23090633_20120403.hdf5', '23090607_20120420.hdf5', '23090594_20160706.hdf5', '23090622_20150105.hdf5', '23090583_20160308.hdf5', '23090637_2

In [54]:
os.listdir('/mnt/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/')

['fold_4',
 'fold_0',
 'fold_2',
 'all',
 'fold_1',
 'plans.pkl',
 'fold_3',
 'gt_niftis']

In [45]:
os.listdir('/mnt/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/3d_fullres/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/')

['fold_4',
 'fold_0',
 'fold_2',
 'all',
 'fold_1',
 'plans.pkl',
 'fold_3',
 'gt_niftis']

In [42]:
# Rename model..
# os.rename('/mnt/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/Task555_PETCT', '/mnt/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/all_Task555_PETCT')

In [57]:
import shutil
# Copy model..
# shutil.copytree('/mnt/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/', '/mnt/backup/temp_submission/models/nnUNet/2d')
# shutil.copytree('/tf/submission/submitted/21-10-27_13:29:02-models/', '/tf/temp_submission/models')
# shutil.copytree('/tf/backup/nnUNet', '/tf/temp_submission/models/module')
shutil.copytree('/tf/backup/temp_submission/', '/tf/temp/')

'/tf/temp/'

In [24]:
os.listdir('/mnt/')

['utils', 'backup', 'dataset', 'submission', 'temp']

In [31]:
(os.listdir('/mnt/'))

['utils', 'backup', 'dataset', 'submission', 'temp']